In [1]:
'''
A notebook to run and show demo of various queries
'''

'\nA notebook to run and show demo of various queries\n'

In [2]:
import json

from os.path import join

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [3]:
ES_HOST = 'http://localhost'
ES_PORT = 9202
ES_HOST_STRING = '{}:{}'.format(ES_HOST, ES_PORT)

INDEX_NAME = 'movies'
TYPE_NAME = 'marvel'

In [4]:
mapping = '''{
  "mappings": {
    "marvel": {
      "properties": {
        "title": {
          "type": "text"
        },
        "year": {
          "type": "long"
        },
        "directors": {
          "type": "text"
        },
        "imdb_rating": {
          "type": "float"
        },
        "summary": {
          "type": "text"
        },
        "duration": {
          "type": "integer"
        }
      }
    }
  }
}'''

In [5]:
es = Elasticsearch([ES_HOST_STRING])

In [6]:
es.indices.create(index=INDEX_NAME, ignore=400, body=mapping)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies'}

In [7]:
data_file = join('data', 'marvel_movies_released.json')

In [8]:
with open(data_file) as infile:
    data = json.load(infile)

In [9]:
def make_documents(doc, doc_id):
    es_doc = {
        '_op_type': 'index',
        '_index': INDEX_NAME,
        '_type': TYPE_NAME,
        '_id': doc_id,
        '_source': doc
    }
    return es_doc

In [10]:
bulk(es, [make_documents(doc, idx) for idx, doc in enumerate(data)], index=INDEX_NAME, doc_type=TYPE_NAME)

(19, [])

In [11]:
# search for ironman
search_query = '''{
  "query": {
    "match": {
      "title": "iron man"
    }
  }
}'''

In [12]:
response = es.search(index=INDEX_NAME, doc_type=TYPE_NAME, body=search_query, _source_include=['title'])

In [14]:
def pretty_print_es_response(es_response):
    pretty_response = []
    hits = es_response.get('hits', []).get('hits', [])
    for hit in hits:
        source = hit.get('_source')
        source['id'] = hit.get('_id')
        pretty_response.append(source)
    return pretty_response

In [15]:
pretty_print_es_response(response)

[{'title': 'Iron Man 3', 'id': '12'},
 {'title': 'Iron Man', 'id': '18'},
 {'title': 'Iron Man 2', 'id': '16'},
 {'title': 'Spider-Man: Homecoming', 'id': '2'},
 {'title': 'Ant-Man', 'id': '7'}]

In [16]:
# search for ironman on title and summary field
search_query = '''{
  "query": {
    "multi_match": {
      "query": "iron man",
      "fields": ["title", "summary"]
    }
  }
}'''

In [19]:
response = es.search(index=INDEX_NAME, doc_type=TYPE_NAME, body=search_query, _source_include=['title', 'summary'])

In [20]:
pretty_print_es_response(response)

[{'summary': "With the world now aware of his identity as Iron Man, Tony Stark must contend with both his declining health and a vengeful mad man with ties to his father's legacy.",
  'title': 'Iron Man 2',
  'id': '16'},
 {'summary': "When Tony Stark's world is torn apart by a formidable terrorist called the Mandarin, he starts an odyssey of rebuilding and retribution.",
  'title': 'Iron Man 3',
  'id': '12'},
 {'summary': "Political involvement in the Avengers' activities causes a rift between Captain America and Iron Man.",
  'title': 'Captain America: Civil War',
  'id': '6'},
 {'summary': 'After being held captive in an Afghan cave, billionaire engineer Tony Stark creates a unique weaponized suit of armor to fight evil.',
  'title': 'Iron Man',
  'id': '18'},
 {'summary': 'Peter Parker balances his life as an ordinary high school student in Queens with his superhero alter-ego Spider-Man, and finds himself on the trail of a new menace prowling the skies of New York City.',
  'title

In [21]:
# Notice how movie with id `6` appears though it does not contain Iron man in the title.

In [22]:
# lets search for iron man again but this with a boost on summary field
search_query = '''{
  "query": {
    "multi_match": {
      "query": "iron man",
      "fields": ["title", "summary^2"]
    }
  },
  "_source": ["id", "title", "summary"]
}'''

In [23]:
response = es.search(index=INDEX_NAME, doc_type=TYPE_NAME, body=search_query, _source_include=['title', 'summary'])
pretty_print_es_response(response)

[{'summary': "With the world now aware of his identity as Iron Man, Tony Stark must contend with both his declining health and a vengeful mad man with ties to his father's legacy.",
  'title': 'Iron Man 2',
  'id': '16'},
 {'summary': "Political involvement in the Avengers' activities causes a rift between Captain America and Iron Man.",
  'title': 'Captain America: Civil War',
  'id': '6'},
 {'summary': "When Tony Stark's world is torn apart by a formidable terrorist called the Mandarin, he starts an odyssey of rebuilding and retribution.",
  'title': 'Iron Man 3',
  'id': '12'},
 {'summary': 'After being held captive in an Afghan cave, billionaire engineer Tony Stark creates a unique weaponized suit of armor to fight evil.',
  'title': 'Iron Man',
  'id': '18'},
 {'summary': 'Peter Parker balances his life as an ordinary high school student in Queens with his superhero alter-ego Spider-Man, and finds himself on the trail of a new menace prowling the skies of New York City.',
  'title

In [24]:
# notice how the ranking changes of doc with id `6` and id `12` when a boost factor is given to `summary` field

In [25]:
# search for captan amrica
search_query = '''{
  "query": {
    "match": {
      "title": {
        "query": "captan amrica",
        "fuzziness": 2
      }
    }
  }
}'''

In [26]:
response = es.search(index=INDEX_NAME, doc_type=TYPE_NAME, body=search_query, _source_include=['title', 'summary'])
pretty_print_es_response(response)

[{'summary': 'As Steve Rogers struggles to embrace his role in the modern world, he teams up with a fellow Avenger and S.H.I.E.L.D agent, Black Widow, to battle a new threat from history: an assassin known as the Winter Soldier.',
  'title': 'Captain America: The Winter Soldier',
  'id': '10'},
 {'summary': "Political involvement in the Avengers' activities causes a rift between Captain America and Iron Man.",
  'title': 'Captain America: Civil War',
  'id': '6'},
 {'summary': "Steve Rogers, a rejected military soldier transforms into Captain America after taking a dose of a 'Super-Soldier serum'. But being Captain America comes at a price as he attempts to take down a war monger and a terrorist organization.",
  'title': 'Captain America: The First Avenger',
  'id': '14'}]

In [28]:
# notice how fuzzy value of 2 takes care of captan(captain) and amrica(america)

In [29]:
# search for iron man as a phrase
search_query = '''{
  "query": {
    "match_phrase": {
      "title": "iron man"
    }
  }
}'''

In [30]:
response = es.search(index=INDEX_NAME, doc_type=TYPE_NAME, body=search_query, _source_include=['title', 'summary'])
pretty_print_es_response(response)

[{'summary': "When Tony Stark's world is torn apart by a formidable terrorist called the Mandarin, he starts an odyssey of rebuilding and retribution.",
  'title': 'Iron Man 3',
  'id': '12'},
 {'summary': 'After being held captive in an Afghan cave, billionaire engineer Tony Stark creates a unique weaponized suit of armor to fight evil.',
  'title': 'Iron Man',
  'id': '18'},
 {'summary': "With the world now aware of his identity as Iron Man, Tony Stark must contend with both his declining health and a vengeful mad man with ties to his father's legacy.",
  'title': 'Iron Man 2',
  'id': '16'}]

In [31]:
# notice how only three movies appear, in comparison to 5 when iron man was searched as a match query

In [32]:
# search for movies of a particular year
search_query = '''{
  "query": {
    "bool": {
      "filter": {
        "term": {
          "year": 2018
        }
      }
    }
  }
}'''

In [34]:
response = es.search(index=INDEX_NAME, doc_type=TYPE_NAME, body=search_query, _source_include=['title', 'summary', 'year'])
pretty_print_es_response(response)

[{'summary': "T'Challa, the King of Wakanda, rises to the throne in the isolated, technologically advanced African nation, but his claim is challenged by a vengeful outsider who was a childhood victim of T'Challa's father's mistake.",
  'year': '2018',
  'title': 'Black Panther',
  'id': '0'},
 {'summary': 'The Avengers and their allies must be willing to sacrifice all in an attempt to defeat the powerful Thanos before his blitz of devastation and ruin puts an end to the universe.',
  'year': '2018',
  'title': 'Avengers: Infinity War',
  'id': '1'}]

In [35]:
# lets search for movies within a particular time period
search_query = '''{
  "query": {
    "bool": {
      "filter": {
        "range": {
          "year": {
            "gte": 2015,
            "lte": 2016
          }
        }
      }
    }
  }
}'''

In [36]:
response = es.search(index=INDEX_NAME, doc_type=TYPE_NAME, body=search_query, _source_include=['title', 'summary', 'year'])
pretty_print_es_response(response)

[{'summary': 'While on a journey of physical and spiritual healing, a brilliant neurosurgeon is drawn into the world of the mystic arts.',
  'year': '2016',
  'title': 'Doctor Strange',
  'id': '5'},
 {'summary': "When Tony Stark and Bruce Banner try to jump-start a dormant peacekeeping program called Ultron, things go horribly wrong and it's up to Earth's mightiest heroes to stop the villainous Ultron from enacting his terrible plan.",
  'year': '2015',
  'title': 'Avengers: Age of Ultron',
  'id': '8'},
 {'summary': "Political involvement in the Avengers' activities causes a rift between Captain America and Iron Man.",
  'year': '2016',
  'title': 'Captain America: Civil War',
  'id': '6'},
 {'summary': 'Armed with a super-suit with the astonishing ability to shrink in scale but increase in strength, cat burglar Scott Lang must embrace his inner hero and help his mentor, Dr. Hank Pym, plan and pull off a heist that will save the world.',
  'year': '2015',
  'title': 'Ant-Man',
  'id'